<a href="https://colab.research.google.com/github/yj9889/ESAA2/blob/main/220328%EC%95%99%EC%83%81%EB%B8%94%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 핸즈온 머신러닝 - 07. 앙상블 학습과 랜덤 포레스트

* 앙상블: 일련의 예측기로 예측을 수집해 가장 좋은 모델 하나보다 더 좋은 예측을 얻는 방법.

> ## 7.1 투표 기반 분류기
* 직접 투표 분류기: 각 분류기의 예측을 모아 가장 많이 선택된 클래스 예측(다수결 투표)
  * 앙상블은 예측기가 가능한 서로 독립적일 때 최고의 성능

In [3]:
# 사이킷런 다수 분류기
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# 선형 회귀 
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
# 랜덤 포레스트
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
# 서포트 벡터 머신
svm_clf = SVC(gamma="scale", random_state=42)

# 다수결 분류기: 직접 투표 방식
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='hard')

# 다수결 분류기 학습
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [5]:
from sklearn.metrics import accuracy_score

# 분류기 별 정확도 측정
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__+':\t', accuracy_score(y_test, y_pred))

LogisticRegression:	 0.864
RandomForestClassifier:	 0.896
SVC:	 0.896
VotingClassifier:	 0.912


* 간접 투표: 모든 분류기가 클래스의 예측을 확인할 수 있으면(predict_proba() 메서드가 있으면), 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스 예측 가능
  * *voting='hard'* -> *voting='soft'*
  * SVC 기본값) *probability=True'*: SVC에서 predict_proba 사용 가능

> ## 7.2 배깅과 페이스팅
같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것
* 배깅: 훈련 세트에서 중복을 허용하여 샘플링하는 방식
* 페이스팅: 훈련 세트에서 중복을 허용하지 않고 샘플링하는 방식 
  * 분류) 통계적 최빈값
  * 회귀) 평균
>> ## 7.2.1 사이킷런의 배깅과 페이스팅
* 분류) *BaggingClassifier* 
* 회귀) *BaggingRegressor*

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), 
    n_estimators=500,
    max_samples=100, 
    # bootstrap=True 가 기본값임.
#     bootstrap=True,
    n_jobs=-1,
    random_state=42)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [7]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

0.904


In [8]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)

print(accuracy_score(y_test, y_pred_tree))

0.856


>> ## 7.2.2 oob 평가
* 배깅을 사용하면 한 번도 선택되지 못하는 샘플이 존재, 뽑히지 않을 확률은 37% 정도
* oob샘플: 선택되지 않은 훈련 샘플
  * 사이킷런) BaggingClassifier 에 *oob_score=True*옵션

In [15]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), 
    n_estimators=500, 
    bootstrap=True, 
    n_jobs=-1,
    oob_score=True)
bag_clf.fit(X_train, y_train)

bag_clf.oob_score_

0.9013333333333333

In [14]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.904

In [17]:
bag_clf.oob_decision_function_[:10]

array([[0.42696629, 0.57303371],
       [0.30810811, 0.69189189],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.00549451, 0.99450549],
       [0.0887574 , 0.9112426 ],
       [0.37234043, 0.62765957],
       [0.00505051, 0.99494949],
       [1.        , 0.        ],
       [0.98351648, 0.01648352]])

> ## 7.3 랜덤 배치와 랜덤 서브스페이스
특성을 대상으로 하는 두 파라미터
* max_features: 학습에 사용할 특성 수를 지정
* bootstrap_features: 학습에 사용할 특성을 선택할 때 중복 허용 여부 지정

* 랜덤 패치 방식: 훈련 특성과 샘플을 모두 샘플링
* 랜덤 서브스페이스 방식: 훈련 샘플은 모두 사용하고 특성은 샘플링
  * 훈련 샘플 모두) *bootstrap=False, max_sample=0.1*
  * 특성 샘플링) *bootstrap_features=True, max_features=0.1보다 작게*: 더 다양한 예측기, 편향을 늘리는 대신 분산 낮춤

> ## 7.4 랜덤 포레스트
* BaggingClassifier에 DecisionTreeClassifier를 넣어 만드는 대신, RandomForestClassifier 사용


In [18]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(
    n_estimators=500, # 500개의 의사결정나무 학습
    max_leaf_nodes=16, # 사용되는 의사결정나무의 잎의 수를 16개로 제한
    n_jobs=-1) # 가용 가능한 모든 CPU 사용

rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

In [19]:
bag_clf_auto = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500,
    # max_samples=1.0 이 기본값임.
    max_samples=1.0, 
    # bootstrap=True 가 기본값임.
    bootstrap=True,
    n_jobs=-1)

>> ## 7.4.1 엑스트라 트리
* 익스트림 랜덤 트리: 극단적으로 무작위한 트리의 랜덤 트리
  * 편향 늘지만, 분산 줄음
  * *ExtraTreesClassifier*

>> ## 7.4.2 특성 중요도
* 랜덤 포레스트는 학습에 사용된 특성의 상대적 중요도도 함께 측정한다. 특성 중요도는 해당 특성을 사용한 마디가 평균적으로 불순도를 얼마나 감소시키는지를 측정한다. 즉, 불순도를 많이 줄이면 그만큼 중요도가 커진다.
  * 가중치(각 노드의 연관된 훈련 샘플 수) 평균
* 사이킷런의 RandomForestClassifier는 훈련이 끝난 뒤 특성별 중요도의 전체 합이 1이 되도록 하는 방식으로 특성별 상대적 중요도를 측정한 후 feature_importances_ 속성에 저장

In [20]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.11249225099876375
sepal width (cm) 0.02311928828251033
petal length (cm) 0.4410304643639577
petal width (cm) 0.4233579963547682


> ## 7.5 부스팅
* 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법
>> ## 7.5.1 에이다부스트
* 이전 모델이 과소적합했던 훈련 샘플의 가중치를 높여 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰짐.
  * cf. 경사 하강법: 비용 함수를 최소화하기 위해 한 예측기의 모델 파라미터 조정 <-> 에이다부스트: 앙상블에 예측기 추가

In [21]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), 
    n_estimators=200,
    algorithm="SAMME.R", 
    learning_rate=0.5, 
    random_state=42)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200, random_state=42)

> ## 7.5.2 그레이디언트 부스팅
* 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 **잔여 오차**에 새로운 예측기 학습시킴.

In [22]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

# 의사결정나무 회귀모델을 이용하여 학습
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [24]:
# 예측기의 예측 결과와의 오차에 대한 레이블을 대상으로 의사결정나무 회귀모델 학습
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [25]:
# 예측기의 예측 결과와의 오차에 대한 레이블을 대상으로 의사결정나무 회귀모델 학습
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [26]:
# 새로운 샘플
X_new = np.array([[0.8]])
# 모든 예측값 더하기
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

* *GradientBoostingRegressor*: n_estimators, max_depth, min_samples_leaf 등으로 조절

In [27]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

* **축소**: *learning_rate*를 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 예측 성능이 좋아짐.

* 최적의 트리 찾기: 조기 종료 기법
  * staged_predict()
  * *warm_start=True*: 사이킷런이 fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 함.


In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 이전 학습 내용을 지우고 새로 훈련하기 위해 훈련 세트 섞기. 
# random_state를 다른 값으로 지정.
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)

# GBRT 모델 설정 및 훈련
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt.fit(X_train, y_train)

# staged_predict()에 의해 생성된 반복자를 활용하여
# 각 단계별 MSE 수집 후 최소값을 갖는 인덱스 확인
errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

# 최적의 의사결정나무 수를 이용하여 새로 학습
gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=56, random_state=42)

* 확률적 그레디언트 부스팅: 훈련 샘플의 비율을 지정할 수 있는 subsample 사용하여 적은 비율의 훈련샘플 사용하면 편향 높아지는 대신 분산 낮아짐

In [29]:
import xgboost
xgb_reg = xgboost.XGBRegressor(random_state=42)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

[06:40:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [30]:
xgb_reg.fit(X_train, y_train, eval_set = [(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[06:41:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.286719
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.258221
[2]	validation_0-rmse:0.232634
[3]	validation_0-rmse:0.210526
[4]	validation_0-rmse:0.190232
[5]	validation_0-rmse:0.172196
[6]	validation_0-rmse:0.156394
[7]	validation_0-rmse:0.142241
[8]	validation_0-rmse:0.129789
[9]	validation_0-rmse:0.118752
[10]	validation_0-rmse:0.108388
[11]	validation_0-rmse:0.100155
[12]	validation_0-rmse:0.09208
[13]	validation_0-rmse:0.084791
[14]	validation_0-rmse:0.078699
[15]	validation_0-rmse:0.073248
[16]	validation_0-rmse:0.069391
[17]	validation_0-rmse:0.066277
[18]	validation_0-rmse:0.063458
[19]	validation_0-rmse:0.060326
[20]	validation_0-rmse:0.0578
[21]	validation_0-rmse:0.055643
[22]	validation_0-rmse:0.053943
[23]	validation_0-rmse:0.053138
[24]	validation_0-rmse:0.052415
[25]	validation_0-rmse

> ## 7.6 스태킹
* 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수(like 직접 투표) 대신 취합하는 모델을 훈련시킴.